#### **Unconventionality Measure (own / others)**

In [0]:
import pandas as pd
import pickle
import glob
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from collections import Counter

#### **1. Read tables and merging**

In [3]:
director_films = pd.read_csv("imdb-director-films.csv", encoding="iso-8859-1")
scraping = pd.read_csv("scraping.csv", encoding="iso-8859-1")
director_films.head(5)

,imdbcrewcastname,imdbcrewcastlink,year,imdbfilmtitle,role,imdbfilmlink
0,A.R. Murugadoss,http://www.imdb.com/name/nm1436693/,2017.0,Spyder,Director,http://www.imdb.com/title/tt6522546/
1,A.R. Murugadoss,http://www.imdb.com/name/nm1436693/,2001.0,Dheena,Director,http://www.imdb.com/title/tt0807944/
2,A.R. Murugadoss,http://www.imdb.com/name/nm1436693/,2005.0,Ghajini,Director,http://www.imdb.com/title/tt0449951/
3,A.R. Murugadoss,http://www.imdb.com/name/nm1436693/,2006.0,Stalin: Man for the Society,Director,http://www.imdb.com/title/tt0366180/
4,A.R. Murugadoss,http://www.imdb.com/name/nm1436693/,2008.0,Ghajini,Director,http://www.imdb.com/title/tt1166100/


In [0]:
director_films = director_films[director_films['role'] == "Director"]

In [5]:
scraping.columns

Index(['Unnamed: 0', 'year', 'bomrank', 'remove', 'bomtitle', 'imdblink',
       'bomlink', 'studio', 'totalusgross$', 'totaltheater', 'openingusgross$',
       'openingtheaters', 'opendate', 'closedate', 'critic ratings_avg',
       'critic rating_var', 'user_avg', 'user_var', 'critic_ratings_comments',
       'critic_ratings_list', 'ALL_numCritics', 'ALL_meanCritics',
       'ALL_varCritics', 'ALL_listCritics'],
      dtype='object')

In [6]:
result_set = scraping[['year', 'bomtitle', 'imdblink', 'bomlink', 
                      'opendate', 'closedate']]
# change column name to match
result_set = result_set.rename(columns={"bomtitle": "imdbfilmtitle"})
print(result_set.shape)
print(len(result_set['imdbfilmtitle'].unique()))
result_set.head(5)

(2400, 6)
2400


,year,imdbfilmtitle,imdblink,bomlink,opendate,closedate
0,2004,Shrek 2,http://www.imdb.com/title/tt0298148/,http://www.boxofficemojo.com/movies/?id=shrek2...,19-May,25-Nov
1,2004,Spider-Man 2,http://www.imdb.com/title/tt0316654/,http://www.boxofficemojo.com/movies/?id=spider...,30-Jun,19-Dec
2,2004,The Passion of the Christ,http://www.imdb.com/title/tt0335345/,http://www.boxofficemojo.com/movies/?id=passio...,25-Feb,29-Jul
3,2004,Meet the Fockers,http://www.imdb.com/title/tt0290002/,http://www.boxofficemojo.com/movies/?id=meetth...,22-Dec,16-Jun
4,2004,The Incredibles,http://www.imdb.com/title/tt0317705/,http://www.boxofficemojo.com/movies/?id=incred...,5-Nov,14-Apr


In [7]:
result_set = pd.merge(result_set, director_films[["imdbfilmtitle", "imdbcrewcastname"]], on="imdbfilmtitle")
print("Unique available movies: ", len(result_set['imdbfilmtitle'].unique()))
print("Unique directors: ", len(result_set['imdbcrewcastname'].unique()))
print(len(result_set))
result_set.head(5)

Unique available movies:  1961
Unique directors:  1255
2359


,year,imdbfilmtitle,imdblink,bomlink,opendate,closedate,imdbcrewcastname
0,2004,Shrek 2,http://www.imdb.com/title/tt0298148/,http://www.boxofficemojo.com/movies/?id=shrek2...,19-May,25-Nov,Andrew Adamson
1,2004,Shrek 2,http://www.imdb.com/title/tt0298148/,http://www.boxofficemojo.com/movies/?id=shrek2...,19-May,25-Nov,Conrad Vernon
2,2004,Shrek 2,http://www.imdb.com/title/tt0298148/,http://www.boxofficemojo.com/movies/?id=shrek2...,19-May,25-Nov,Kelly Asbury
3,2004,Spider-Man 2,http://www.imdb.com/title/tt0316654/,http://www.boxofficemojo.com/movies/?id=spider...,30-Jun,19-Dec,Sam Raimi
4,2004,The Passion of the Christ,http://www.imdb.com/title/tt0335345/,http://www.boxofficemojo.com/movies/?id=passio...,25-Feb,29-Jul,Mel Gibson


#### **2. Start collecting descriptors info**

In [0]:
# package use
import requests
import urllib
import urllib.request
import time
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from IPython.core.display import HTML

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import copy
from collections import Counter

In [9]:
result_set = pd.read_csv("Film-director(sample only).csv")
result_set = result_set[['year', 'imdbcrewcastlink', 'imdbfilmtitle', 'imdbfilmlink', 'bomtitle', 'bomlink']]
result_set = result_set[result_set['year'] >= 2004].reset_index(drop=True)
print(result_set.shape)
result_set.head(5)

(2597, 6)


,year,imdbcrewcastlink,imdbfilmtitle,imdbfilmlink,bomtitle,bomlink
0,2005,http://www.imdb.com/name/nm4588077/,Fantastic Four,http://www.imdb.com/title/tt0120667/,Fantastic Four (2005),http://www.boxofficemojo.com/movies/?id=fantas...
1,2005,http://www.imdb.com/name/nm1103162/,Fantastic Four,http://www.imdb.com/title/tt0120667/,Fantastic Four (2005),http://www.boxofficemojo.com/movies/?id=fantas...
2,2005,http://www.imdb.com/name/nm0425843/,Corpse Bride,http://www.imdb.com/title/tt0121164/,Tim Burton's Corpse Bride,http://www.boxofficemojo.com/movies/?id=corpse...
3,2005,http://www.imdb.com/name/nm0000318/,Corpse Bride,http://www.imdb.com/title/tt0121164/,Tim Burton's Corpse Bride,http://www.boxofficemojo.com/movies/?id=corpse...
4,2005,http://www.imdb.com/name/nm0000184/,Star Wars: Episode III - Revenge of the Sith,http://www.imdb.com/title/tt0121766/,Star Wars: Episode III - Revenge of the Sith,http://www.boxofficemojo.com/movies/?id=starwa...


In [10]:
result_set['descriptor1'] = np.nan
result_set['descriptor2'] = np.nan
result_set['descriptor3'] = np.nan
result_set = result_set.reset_index(drop=True)
result_set.head(1)

,year,imdbcrewcastlink,imdbfilmtitle,imdbfilmlink,bomtitle,bomlink,descriptor1,descriptor2,descriptor3
0,2005,http://www.imdb.com/name/nm4588077/,Fantastic Four,http://www.imdb.com/title/tt0120667/,Fantastic Four (2005),http://www.boxofficemojo.com/movies/?id=fantas...,NaN,NaN,NaN


In [0]:
# fill in descriptors 
for idx in range(len(result_set)):
  url = result_set['imdbfilmlink'][idx]
  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser")
  select = soup.find_all("div", {"class": "subtext"})[0]
  filter_ = re.findall(r'genres=(.*?)&amp', str(select))
  # find available descriptors to fill
  if len(filter_) > 0:
    for i in range(len(filter_)):
      result_set['descriptor'+str(i+1)][idx] = filter_[i]
  if idx%200 == 0:
    print(idx)

In [0]:
result_set.to_csv("result_set_descriptor.csv")

In [0]:
result_set.head(5)

,year,imdbcrewcastlink,imdbfilmtitle,imdbfilmlink,bomtitle,bomlink,descriptor1,descriptor2,descriptor3
0,2005,http://www.imdb.com/name/nm4588077/,Fantastic Four,http://www.imdb.com/title/tt0120667/,Fantastic Four (2005),http://www.boxofficemojo.com/movies/?id=fantas...,action,adventure,fantasy
1,2005,http://www.imdb.com/name/nm1103162/,Fantastic Four,http://www.imdb.com/title/tt0120667/,Fantastic Four (2005),http://www.boxofficemojo.com/movies/?id=fantas...,action,adventure,fantasy
2,2005,http://www.imdb.com/name/nm0425843/,Corpse Bride,http://www.imdb.com/title/tt0121164/,Tim Burton's Corpse Bride,http://www.boxofficemojo.com/movies/?id=corpse...,animation,drama,family
3,2005,http://www.imdb.com/name/nm0000318/,Corpse Bride,http://www.imdb.com/title/tt0121164/,Tim Burton's Corpse Bride,http://www.boxofficemojo.com/movies/?id=corpse...,animation,drama,family
4,2005,http://www.imdb.com/name/nm0000184/,Star Wars: Episode III - Revenge of the Sith,http://www.imdb.com/title/tt0121766/,Star Wars: Episode III - Revenge of the Sith,http://www.boxofficemojo.com/movies/?id=starwa...,action,adventure,fantasy


In [11]:
result_set = pd.read_csv("result_set_descriptor.csv")
result_set.head(1)

,year,imdbcrewcastlink,imdbfilmtitle,imdbfilmlink,bomtitle,bomlink,descriptor1,descriptor2,descriptor3
0,2005,http://www.imdb.com/name/nm4588077/,Fantastic Four,http://www.imdb.com/title/tt0120667/,Fantastic Four (2005),http://www.boxofficemojo.com/movies/?id=fantas...,action,adventure,fantasy


#### **3. Count unconventionality**

In [0]:
import re
# count number of movies each has
result_set['NumOwnFilms(past five years)'] = np.nan
result_set['NumOfTwoCommonCategoriesOwnFilms(past five years)'] = np.nan

In [0]:
result_set = pd.read_csv("count.csv")

In [0]:
upper_bound = len(result_set)
for idx in range(1310, upper_bound):
  url = result_set['imdbcrewcastlink'][idx]
  # set up year constraints
  year = result_set['year'][idx]
  year_limit = [str(year-i) for i in range(5)]
  descriptor_limit = [result_set['descriptor1'][idx], result_set['descriptor2'][idx], result_set['descriptor3'][idx]]

  # filter all own films
  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser")
  filter_ = soup.find_all("div", id=re.compile("director"))
  number_own_movies = 0 # count recent five years movies
  number_own_movies_common = 0 # count two common descriptors movies
  for item in filter_:
    temp_list = re.findall(r'2[0-9][0-9][0-9]', item.text)
    # count recent five years movies and operate
    if len(temp_list) > 0 and temp_list[0] in year_limit:
      number_own_movies += 1
      # find its url address to extract descriptor
      url_additional = "http://www.imdb.com" + re.search(r'href="(.*?)"', str(item)).group(1)
      response_additional = requests.get(url_additional)
      soup_additional = BeautifulSoup(response_additional.text, "html.parser")
      select_additional = soup_additional.find_all("div", {"class": "subtext"})[0]
      filter_additional = re.findall(r'genres=(.*?)&amp', str(select_additional))
      descriptor_count = 0 # count common descriptor for each film
      if len(filter_additional) > 0:
        for i in range(len(filter_additional)):
          if filter_additional[i] in descriptor_limit:
            descriptor_count += 1
      if descriptor_count >= 2:
        number_own_movies_common += 1
  # update info in result_set
  result_set['NumOwnFilms(past five years)'][idx] = number_own_movies
  result_set['NumOfTwoCommonCategoriesOwnFilms(past five years)'][idx] = number_own_movies_common

  if idx%10 == 0:
    print(idx)

In [15]:
result_set.head(10)

,year,imdbcrewcastlink,imdbfilmtitle,imdbfilmlink,bomtitle,bomlink,descriptor1,descriptor2,descriptor3,NumOwnFilms(past five years),NumOfTwoCommonCategoriesOwnFilms(past five years)
0,2005,http://www.imdb.com/name/nm4588077/,Fantastic Four,http://www.imdb.com/title/tt0120667/,Fantastic Four (2005),http://www.boxofficemojo.com/movies/?id=fantas...,action,adventure,fantasy,3.0,1.0
1,2005,http://www.imdb.com/name/nm1103162/,Fantastic Four,http://www.imdb.com/title/tt0120667/,Fantastic Four (2005),http://www.boxofficemojo.com/movies/?id=fantas...,action,adventure,fantasy,3.0,1.0
2,2005,http://www.imdb.com/name/nm0425843/,Corpse Bride,http://www.imdb.com/title/tt0121164/,Tim Burton's Corpse Bride,http://www.boxofficemojo.com/movies/?id=corpse...,animation,drama,family,1.0,1.0
3,2005,http://www.imdb.com/name/nm0000318/,Corpse Bride,http://www.imdb.com/title/tt0121164/,Tim Burton's Corpse Bride,http://www.boxofficemojo.com/movies/?id=corpse...,animation,drama,family,4.0,1.0
4,2005,http://www.imdb.com/name/nm0000184/,Star Wars: Episode III - Revenge of the Sith,http://www.imdb.com/title/tt0121766/,Star Wars: Episode III - Revenge of the Sith,http://www.boxofficemojo.com/movies/?id=starwa...,action,adventure,fantasy,2.0,2.0
5,2004,http://www.imdb.com/name/nm0868219/,Hellboy,http://www.imdb.com/title/tt0167190/,Hellboy,http://www.boxofficemojo.com/movies/?id=hellbo...,action,adventure,fantasy,3.0,1.0
6,2004,http://www.imdb.com/name/nm0000408/,Thunderbirds,http://www.imdb.com/title/tt0167456/,Thunderbirds,http://www.boxofficemojo.com/movies/?id=thunde...,action,adventure,comedy,5.0,2.0
7,2008,http://www.imdb.com/name/nm0002044/,The Bank Job,http://www.imdb.com/title/tt0200465/,The Bank Job,http://www.boxofficemojo.com/movies/?id=bankjo...,crime,thriller,NaN,2.0,1.0
8,2004,http://www.imdb.com/name/nm0001317/,Exorcist: The Beginning,http://www.imdb.com/title/tt0204313/,Exorcist: The Beginning,http://www.boxofficemojo.com/movies/?id=exorci...,horror,mystery,thriller,4.0,2.0
9,2006,http://www.imdb.com/name/nm0190859/,Children of Men,http://www.imdb.com/title/tt0206634/,Children of Men,http://www.boxofficemojo.com/movies/?id=childr...,drama,thriller,NaN,3.0,1.0


In [0]:
result_set.to_csv("temp.csv")

In [0]:
# have to filter out current movie by minus 1 from common films count column
# have to count if there is only one available descriptor as reference